# Data utilities
> A set of convenience functions for handling data

In [1]:
#| default_exp data_utils

In [2]:
#| export
import fastcore.test
from nbdev.showdoc import *
import nptyping
import numpy as np
import pandas

In [3]:
#| export
dropped_items_warning=f"""Several items in `results` are not suitable for conversion to
a dataframe. This may be because they are not numpy arrays or because they
are not the same size as the other items. 

The following items were dropped: """
def results_to_dataframe_egt(results:dict, # A dictionary containing items from `ModelTypeEGT`.
                             suppress:bool=True, # Supress the dropped items warning
                            ):
    """Convert results to a dataframe, keeping only items which are valid for
    a dataframe to have."""
    flat_results = {k:v
                    for k,v in results.items()
                    if (isinstance(v, np.ndarray)
                        and not v.ndim > 1)}
    for i, strategy in enumerate(results.get('strategy_set', [])):
        if "ergodic" in list(results.keys()):
            flat_results[strategy + "_frequency"] = results['ergodic'][:,i]
    dropped_items = [k for k in results.keys() if k not in flat_results]
    if (len(dropped_items)>0 and not suppress):
        print(f"{dropped_items_warning} {dropped_items}")
    return pandas.DataFrame(flat_results)    

In [4]:
#| export
def process_dsair_data(data):
    """Process DSAIR model results dataframe."""
    data['pr'] = np.round(1 - data['p'].values, 2)
    data['s'] = np.round(data['s'].values, 2)
    return data

In [5]:
#| hide
import nbdev; nbdev.nbdev_export()